### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from IPython.display import display, HTML
from matplotlib.cm import plasma
from matplotlib.colors import Normalize

from src.rft_main import *


### Inputs

In [2]:
### INPUTS ###
inputs = {
    "model": "CylinderRough",
    "bulk_density": 1310,
    "friction_material": 0.21,
    "firction_surface": 0.4,
    "friction_type": "coefficient",
    "gravity": 9.81,
    "linear_velocity": 0.1,
    "direction_angle_xz_deg": -90,
    "direction_angle_y_deg": -90,
    "rotation": True,
    "angular_velocity": np.array([0, 0, -2 * np.pi]),
    "start_depth": 0,
    "end_depth": 125,
    "step_size": 5,
}
colormap = "Viridis"


In [3]:
results = run_rft(**inputs)

point_list = results["point_list"]
normal_list = results["normal_list"]
area_list = results["area_list"]
depth_list = results["depth_list"]
object_width_x = results["object_width_x"]
object_width_y = results["object_width_y"]
object_height = results["object_height"]
vertices = results["vertices"]
faces = results["faces"]
trg = results["trg"]
movement = results["movement"]
z_local = results["z_local"]
r_local = results["r_local"]
theta_local = results["theta_local"]
alpha_generic = results["alpha_generic"]
alpha_generic_n = results["alpha_generic_n"]
alpha_generic_t = results["alpha_generic_t"]
alpha = results["alpha"]
depth = results["depth"]
forces = results["forces"]
pressures = results["pressures"]
force_x = results["force_x"]
force_y = results["force_y"]
force_z = results["force_z"]
resultant = results["resultant"]
torques = results["torques"]
torque_x = results["torque_x"]
torque_y = results["torque_y"]
torque_z = results["torque_z"]
resultant_torque = results["resultant_torque"]
result_matrix = results["result_matrix"]

html_content = f"""
<div style="
    background-color: #1f2c56;
    padding: 20px;
    border-radius: 10px;
    font-family: 'Helvetica Neue', Helvetica, Arial, sans-serif;
    text-align: center;
    color: #abbbc9;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
    margin: 10px 0;
">
    <h2 style="
        font-size: 24px;
        margin-bottom: 15px;
        font-weight: 300;
        letter-spacing: 1px;
        color: #4ecdc4;
    ">Results</h2>
    <div style="
        font-size: 18px;
        margin: 10px 0;
    "><strong style="color: #ff6b6b;">Force X:</strong> {force_x:.2f} N</div>
    <div style="
        font-size: 18px;
        margin: 10px 0;
    "><strong style="color: #ff6b6b;">Force Y:</strong> {force_y:.2f} N</div>
    <div style="
        font-size: 18px;
        margin: 10px 0;
    "><strong style="color: #ff6b6b;">Force Z:</strong> {force_z:.2f} N</div>
</div>
"""
display(HTML(html_content))


Processed movement from 0 to 125 mm


In [4]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}
</style>


# Plots

## 3D Mesh

In [5]:
# Creating a 3D plot using Plotly
fig = go.FigureWidget()


fig.add_trace(
    go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        colorscale=colormap,
        showscale=False,
        intensity=vertices[:, 2],
        cmin=np.min(vertices[:, 2]),
        cmax=np.max(vertices[:, 2]),
    )
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(point_list[:, 0]),
            y=np.ptp(point_list[:, 1]),
            z=np.ptp(point_list[:, 2]),
        ),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

display(fig)


FigureWidget({
    'data': [{'cmax': 49.969543,
              'cmin': 0.0,
              'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
                             [0.2222222222222222, '#3e4989'], [0.3333333333333333,
                             '#31688e'], [0.4444444444444444, '#26828e'],
                             [0.5555555555555556, '#1f9e89'], [0.6666666666666666,
                             '#35b779'], [0.7777777777777778, '#6ece58'],
                             [0.8888888888888888, '#b5de2b'], [1.0, '#fde725']],
              'i': array([ 42,   0,   0, ..., 672, 672, 672], dtype=int64),
              'intensity': array([24.98477 , 24.98477 , 24.98477 , ..., 26.728682, 23.240858, 26.728682],
                                 dtype=float32),
              'j': array([380, 380, 381, ..., 780, 779, 484], dtype=int64),
              'k': array([  0, 381,  43, ..., 779, 778, 673], dtype=int64),
              'showscale': False,
              'type': 'mesh3d',
 

## Movement

In [6]:
fig = go.FigureWidget()
fig.add_trace(
    go.Cone(
        x=point_list[:, 0],
        y=point_list[:, 1],
        z=point_list[:, 2],
        u=movement[:, 0],
        v=movement[:, 1],
        w=movement[:, 2],
        sizeref=3,
        showscale=False,
    )
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(
            x=np.ptp(point_list[:, 0]),
            y=np.ptp(point_list[:, 1]),
            z=np.ptp(point_list[:, 2]),
        ),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

display(fig)


FigureWidget({
    'data': [{'showscale': False,
              'sizeref': 3,
              'type': 'cone',
              'u': array([-0.9750784 , -0.97416377, -0.97324437, ...,  0.96403446,  0.96403446,
                           0.95514356]),
              'uid': 'e75de1a4-8246-4fa9-b7c7-0f1c0703a270',
              'v': array([-0.00849426, -0.04412649, -0.06188812, ...,  0.15031517,  0.15031517,
                           0.20134536]),
              'w': array([-0.22169792, -0.22148997, -0.22128093, ..., -0.21918693, -0.21918693,
                          -0.21716545]),
              'x': array([  0.6097955,   3.1707752,   4.451265 , ..., -10.914612 , -10.914612 ,
                          -14.756081 ], dtype=float32),
              'y': array([-70., -70., -70., ...,  70.,  70.,  70.], dtype=float32),
              'z': array([ -87.883736,  -89.16501 ,  -87.883736, ..., -116.07168 , -118.634224,
                          -116.07168 ], dtype=float32)}],
    'layout': {'height': 500,
 

## Force Quiver

In [7]:
# Calculating the endpoints and the magnitudes of the vectors
endpoints = point_list - forces * 200
magnitudes = np.linalg.norm(forces, axis=1)
# Normalize the magnitudes to the range [0, 1]
norm = Normalize()
normalized_magnitudes = norm(magnitudes)
# Get colors from a colormap
colors = plasma(normalized_magnitudes)

# Preparing data for plot
x_lines = np.c_[
    point_list[:, 0], endpoints[:, 0], np.full(len(point_list), np.nan)
].flatten()
y_lines = np.c_[
    point_list[:, 1], endpoints[:, 1], np.full(len(point_list), np.nan)
].flatten()
z_lines = np.c_[
    point_list[:, 2], endpoints[:, 2], np.full(len(point_list), np.nan)
].flatten()

# Flatten the colors array and convert to RGB strings
colors = (colors[:, :3] * 255).astype(int)
color_strings = [f"rgb({r},{g},{b})" for r, g, b in colors]
color_strings = np.repeat(
    color_strings, 3
)  # Repeat for each segment of the line (start, end, break)

# Creating a 3D plot using Plotly
line_marker = dict(width=5, color=color_strings)
fig = go.FigureWidget(
    data=[go.Scatter3d(x=x_lines, y=y_lines, z=z_lines, mode="lines", line=line_marker)]
)

# Updating the layout of the plot
fig.update_layout(
    width=600,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=1.75, y=1.5, z=1.25)),
    ),
)

# Show the plot
display(fig)


FigureWidget({
    'data': [{'line': {'color': array(['rgb(12,7,134)', 'rgb(12,7,134)', 'rgb(12,7,134)', ..., 'rgb(81,1,162)',
                                       'rgb(81,1,162)', 'rgb(81,1,162)'], dtype=object),
                       'width': 5},
              'mode': 'lines',
              'type': 'scatter3d',
              'uid': '63105b7d-57a0-4ceb-ba94-54560bb9dfd7',
              'x': array([  0.60979551,   0.60011064,          nan, ..., -14.75608063,
                          -14.01420925,          nan]),
              'y': array([-70.        , -70.02798161,          nan, ...,  70.        ,
                           72.22607034,          nan]),
              'z': array([ -87.88373566,  -87.88934624,           nan, ..., -116.07167816,
                          -116.56411002,           nan])}],
    'layout': {'height': 500,
               'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10},
               'scene': {'aspectmode': 'data',
                         'aspectratio': {'x'

## Force Scatter

In [8]:
magnitudes = np.linalg.norm(pressures, axis=1)

fig = go.Figure()

scatter = go.Scatter3d(
    x=point_list[:, 0],
    y=point_list[:, 1],
    z=point_list[:, 2],
    mode="markers",
    marker=dict(
        size=2,
        color=magnitudes,  # Set color equal to a variable
        colorscale=colormap,  # Choose a colormap
        colorbar=dict(
            title="Pressure [N/mm²]",
            len=0.75,
            thickness=20,
        ),
        opacity=1,
    ),
)

fig.add_trace(scatter)

# Update the layout if needed
fig.update_layout(
    width=700,
    height=500,
    margin=dict(l=10, r=10, t=10, b=10),
    scene=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=2, y=2, z=1.5)),
    ),
)

# Show the plot
display(fig)


## Force Scatter XYZ

In [9]:
# Calculate global color scale limits based on the maximum absolute pressure value
max_abs_pressure = np.max(np.abs(pressures))

fig = make_subplots(
    rows=1,
    cols=3,
    subplot_titles=("X", "Y", "Z"),
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}]],
)

for i in range(3):
    scatter = go.Scatter3d(
        x=point_list[:, 0],
        y=point_list[:, 1],
        z=point_list[:, 2],
        mode="markers",
        marker=dict(
            size=2,
            color=np.abs(pressures[:, i]),  # Set color equal to a variable
            colorscale=colormap,  # Choose a colormap
            cmin=0,  # Set consistent color scale limits
            cmax=max_abs_pressure,  # Set consistent color scale limits
            colorbar=dict(
                title="Pressure [N/mm²]",
                len=0.75,
                thickness=20,
            ),
            opacity=1,
        ),
    )
    fig.add_trace(scatter, row=1, col=i + 1)

fig.update_layout(
    width=1200,  # Adjusted for 3 plots
    height=500,
    margin=dict(l=10, r=10, t=30, b=10),
    scene1=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
    scene2=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
    scene3=dict(
        aspectmode="data",
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(eye=dict(x=3, y=3, z=2)),
    ),
)

display(fig)


In [10]:
fig = make_subplots(
    rows=2,
    cols=3,
    subplot_titles=("X", "Y", "Z"),
    shared_xaxes=True,
    shared_yaxes=True,
    vertical_spacing=0.05,
    horizontal_spacing=0.05,
)

x_axis_range = [
    np.min(result_matrix[:, 0]),
    np.max(result_matrix[:, 0]),
]
y_axis_range = [
    np.min(result_matrix[:, 1:4]) - np.ptp(result_matrix[:, 1:4]) * 0.1,
    np.max(result_matrix[:, 1:4]) + np.ptp(result_matrix[:, 1:4]) * 0.1,
]

for i in range(3):
    scatter = go.Scatter(
        x=result_matrix[:, 0],
        y=result_matrix[:, i + 1],
        line_shape="spline",
        line=dict(color="slateblue", width=2),
    )
    fig.add_trace(scatter, row=1, col=i + 1)
    fig.update_xaxes(range=x_axis_range, row=1, col=i + 1)
    fig.update_yaxes(range=y_axis_range, row=1, col=i + 1, title_text="Force [N]")

y_axis_range = [
    np.min(result_matrix[:, 4:8]) - np.ptp(result_matrix[:, 4:8]) * 0.1,
    np.max(result_matrix[:, 4:8]) + np.ptp(result_matrix[:, 4:8]) * 0.1,
]

for i in range(3):
    scatter = go.Scatter(
        x=result_matrix[:, 0],
        y=result_matrix[:, i + 4],
        line_shape="spline",
        line=dict(color="teal", width=2),
    )
    fig.add_trace(scatter, row=2, col=i + 1)
    fig.update_xaxes(range=x_axis_range, row=2, col=i + 1, title_text="Depth [mm]")
    fig.update_yaxes(range=y_axis_range, row=2, col=i + 1, title_text="Torque [Nm]")

fig.update_layout(
    width=1200,  # Adjusted for 6 plots
    height=600,
    margin=dict(l=10, r=10, t=30, b=10),
    showlegend=False,
)

display(fig)


In [14]:
temperature = np.array([273, 272, 271.6, 271.4, 271.3, 271, 270.8, 270.6, 270.2, 269.5])

unfrozen_water = np.array([1, 0.99, 0.96, 0.9, 0.81, 0.38, 0.15, 0.06, 0.02, 0])

fig = px.line(
    x=temperature,
    y=unfrozen_water,
    width=600,
    height=500,
    text=unfrozen_water,
)
fig.update_xaxes(title_text="Temperature [K]")
fig.update_yaxes(title_text="Unfrozen water [-]")
fig.update_traces(
    mode="lines+markers",
    marker=dict(size=10, color="slateblue"),
    line=dict(color="slateblue", width=2),
)
# make all text bigger
fig.update_layout(
    font=dict(size=18),
    margin=dict(l=10, r=10, t=30, b=10),
    showlegend=False,
)
fig.show()
fig.write_image("plots/unfrozen_water.svg")


In [55]:
freeze_data = np.loadtxt("freeze_data.csv", delimiter=";", skiprows=0)

fig = px.line(
    width=800,
    height=650,
)
fig.add_trace(
    go.Scatter(
        x=freeze_data[:, 0],
        y=freeze_data[:, 1],
        mode="lines",
        name="Center of tunnel",
        line=dict(width=3, color="#636EFA"),
    )
)
fig.add_trace(
    go.Scatter(
        x=freeze_data[:, 0],
        y=freeze_data[:, 2],
        mode="lines",
        name="Between freeze pipes",
        line=dict(width=3, color="#EF553B"),
    )
)
fig.add_trace(
    go.Scatter(
        x=freeze_data[:, 0],
        y=freeze_data[:, 3],
        mode="lines",
        name="Right of tunnel",
        line=dict(width=3, color="#00CC96"),
    )
)
fig.update_yaxes(title_text="Temperature [K]")
fig.update_xaxes(title_text="Time [d]")

fig.update_layout(
    font=dict(size=18),
    margin=dict(l=10, r=10, t=30, b=10),
    showlegend=True,
    legend=dict(
        x=0.65,
        y=0.99,
        traceorder="normal",
        font=dict(size=18),
        bgcolor="rgba(0,0,0,0)",
    ),
    # paper_bgcolor="rgba(0,0,0,0)",
)
fig.show()
fig.write_image("plots/freezing_pipes.svg")
